In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
print(torch.xpu.is_available())  # torch.xpu is the API for Intel GPU support
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import TensorDataset
import torchvision.transforms as transforms
import pathlib
import os
from PIL import Image
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm

True


Packages, take note that line 6 is specific to my Intel Core GPU, please change (and in subsequent code below) to suit your own GPU [Also downloading Pytorch on different GPU setups might be an issue since I assume most will be using Nvidea - of which I am not using ; so please amend ]

**LOADING DATA**

In [ ]:
df = pd.read_csv('../data/train.csv')
df.head()

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750


I have saved all of the datasets under ../data/ though I am not sure how this is replicated on google colab - need to check

In [ ]:
df_test = pd.read_csv('../data/test.csv')
df_test.head()

,sample_id,image_path,target_name
0,ID1001187975__Dry_Clover_g,test/ID1001187975.jpg,Dry_Clover_g
1,ID1001187975__Dry_Dead_g,test/ID1001187975.jpg,Dry_Dead_g
2,ID1001187975__Dry_Green_g,test/ID1001187975.jpg,Dry_Green_g
3,ID1001187975__Dry_Total_g,test/ID1001187975.jpg,Dry_Total_g
4,ID1001187975__GDM_g,test/ID1001187975.jpg,GDM_g


I loaded in the test dataset but I did not run my code on it fyi, incase you were wondering

**DATA MANIPULATION**

In [ ]:
df_wide = df.copy()
df_wide = df_wide.pivot(index = 'image_path', columns = 'target_name', values = 'target').reset_index()
df_wide["cleaned_path"] = df_wide["image_path"].str.replace(r'^train[\\/]', '', regex=True)
df_wide.head()

target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,cleaned_path
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750,ID1011485656.jpg
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000,ID1012260530.jpg
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500,ID1025234388.jpg
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376,ID1028611175.jpg
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605,ID1035947949.jpg


**SET UP FOR CNN / FUNCTIONS FOR CNN**

In [ ]:
image_folder = '../data/train'
assert os.path.exists(image_folder), f"Image folder {image_folder} does not exist."

df_wide["full_path"] = df_wide["cleaned_path"].apply(
    lambda fname: os.path.join(image_folder, fname)
)

df_wide.head(7)

target_name,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,cleaned_path,full_path
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750,ID1011485656.jpg,../data/train\ID1011485656.jpg
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000,ID1012260530.jpg,../data/train\ID1012260530.jpg
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500,ID1025234388.jpg,../data/train\ID1025234388.jpg
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376,ID1028611175.jpg,../data/train\ID1028611175.jpg
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605,ID1035947949.jpg,../data/train\ID1035947949.jpg
5,train/ID1036339023.jpg,23.0755,2.6135,32.1910,57.8800,55.2665,ID1036339023.jpg,../data/train\ID1036339023.jpg
6,train/ID1049634115.jpg,1.5083,3.0167,13.5750,18.1000,15.0833,ID1049634115.jpg,../data/train\ID1049634115.jpg


This merges the picture directory to the excel table

In [ ]:
image_folder_test = '../data/test'
assert os.path.exists(image_folder_test), f"Image folder {image_folder_test} does not exist."


Random check

In [ ]:
test_path = "../data/train\\ID1011485656.jpg"
print("Path:", test_path)
print("Exists?", os.path.exists(test_path))

Path: ../data/train\ID1011485656.jpg
Exists? True


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df_wide, test_size=0.3, random_state=1)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=1)

Train test Val split

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),        # ← better than CenterCrop for training
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # ← adds variety
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

eval_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),        # ← deterministic for val/test
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Tranformation set up for the pictures; to be pre processed before feeding into CNN

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform = None):
        self.dataframe = dataframe
        self.transform = transform
        self.target_columns = [
            'Dry_Clover_g',
            'Dry_Dead_g',
            'Dry_Green_g',
            'Dry_Total_g',
            'GDM_g'
        ]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        img_path = self.dataframe.iloc[index]['full_path']
        image = Image.open(img_path).convert('RGB')

        targets = self.dataframe.iloc[index][self.target_columns].values.astype('float')
        targets = torch.tensor(targets, dtype = torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, targets

Function to Transform the excel table data and picture into meaningful Tensor

In [ ]:
train_dataset = ImageDataset(train_df, transform = train_transform)
val_dataset = ImageDataset(val_df, transform=eval_transform)
test_dataset = ImageDataset(test_df, transform=eval_transform)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

DataLoader for the CNN

In [ ]:
# Get one batch
for images, targets in train_loader:
    print("Image batch shape:", images.shape)      # Should be [batch_size, 3, 224, 224]
    print("Target batch shape:", targets.shape)   # Should be [batch_size, 5]
    print("Image dtype:", images.dtype)           # Should be torch.float32
    print("Target sample:", targets[0])           # Should be 5 numbers
    break

Image batch shape: torch.Size([8, 3, 224, 224])
Target batch shape: torch.Size([8, 5])
Image dtype: torch.float32
Target sample: tensor([10.0400,  0.0000,  0.0000, 10.0400, 10.0400])


TEST

In [ ]:
import torchvision.models as models

#Load pre-model ResNet-18 instead of building a custom from scratch. - may go for 50 for deeper learning

model = models.resnet18(weights='IMAGENET1K_V1')
#replace the original 100 final output with 5
model.fc = nn.Linear(model.fc.in_features, 5)

device = torch.device('xpu' if hasattr(torch, 'xpu') and torch.xpu.is_available() else 'cpu')
model = model.to(device)

**IMPORTANT** - seting up the initial RESNET (need to downalod first, not sure how this is reflected in the collab though) Also in line 9, you will need to amend this for sure since it uses my Intel XPU GPU!

In [ ]:
# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze ONLY the final layer
for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.MSELoss()

optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)
# above only fine tunes the Final layer, bottom fine-tunes the entire network

# optimizer = optim.Adam(model.parameters(), lr=1e-4)

Freezes learning for all parameters except the last layer - since the first layer are the pre-trained RESNET.
Used MSE as the Loss function
Used Adam as the Optimisation function

In [ ]:
# # Get a sample batch
# sample_images, sample_targets = next(iter(train_loader))
# sample_images, sample_targets = sample_images.to(device), sample_targets.to(device)

# # Forward pass
# outputs = model(sample_images)
# print("Model output shape:", outputs.shape)  # Should be [batch_size, 5]
# print("Sample prediction:", outputs[0].detach().cpu().numpy())
# print("Sample target:", sample_targets[0].cpu().numpy())

Model output shape: torch.Size([8, 5])
Sample prediction: [ 0.00971942  1.9343879  -1.4614012  -0.02772569  0.30146357]
Sample target: [0.  1.9 2.4 4.3 2.4]


In [ ]:
def train_regression(model, num_epochs, train_dl, valid_dl, loss_fn, optimizer, device):
    """
    Train a regression model and return loss histories.

    Returns:
        (loss_hist_train, loss_hist_valid)
    """
    loss_hist_train = [0.0] * num_epochs
    loss_hist_valid = [0.0] * num_epochs

    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            # Move to device (XPU/CPU)
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate total loss (not scaled by batch size here)
            loss_hist_train[epoch] += loss.item()

        # Average loss per batch
        loss_hist_train[epoch] /= len(train_dl)

        # Validation phase
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += loss.item()

        loss_hist_valid[epoch] /= len(valid_dl)

        print(f'Epoch {epoch+1:2d}/{num_epochs} | '
              f'Train Loss: {loss_hist_train[epoch]:.6f} | '
              f'Val Loss: {loss_hist_valid[epoch]:.6f}')

    return loss_hist_train, loss_hist_valid

Function for the regression model

In [ ]:
num_epochs = 20
train_losses, val_losses = train_regression(
    model, num_epochs, train_loader, val_loader,
    criterion, optimizer, device
)

Epoch  1/20 | Train Loss: 1122.382999 | Val Loss: 933.453413
Epoch  2/20 | Train Loss: 635.610828 | Val Loss: 664.177418
Epoch  3/20 | Train Loss: 499.006638 | Val Loss: 543.191275
Epoch  4/20 | Train Loss: 446.954224 | Val Loss: 507.985044
Epoch  5/20 | Train Loss: 433.573085 | Val Loss: 488.657159
Epoch  6/20 | Train Loss: 426.448534 | Val Loss: 485.768921
Epoch  7/20 | Train Loss: 428.725689 | Val Loss: 476.104475
Epoch  8/20 | Train Loss: 421.744473 | Val Loss: 462.091687
Epoch  9/20 | Train Loss: 412.791092 | Val Loss: 466.802404
Epoch 10/20 | Train Loss: 407.008443 | Val Loss: 453.233985
Epoch 11/20 | Train Loss: 407.269029 | Val Loss: 455.514202
Epoch 12/20 | Train Loss: 402.510343 | Val Loss: 433.454459
Epoch 13/20 | Train Loss: 404.323227 | Val Loss: 445.028138
Epoch 14/20 | Train Loss: 408.638635 | Val Loss: 426.674400
Epoch 15/20 | Train Loss: 389.568279 | Val Loss: 433.081437
Epoch 16/20 | Train Loss: 359.156310 | Val Loss: 431.545820
Epoch 17/20 | Train Loss: 443.165263 | 

In [ ]:
torch.save(model.state_dict(), "final_model.pth")
print("Model saved to final_model.pth")

Model saved to final_model.pth


Saving the model parameters for easy replication in the future

In [ ]:
import numpy as np

def calculate_global_weighted_r2(preds, targets):
    """
    Calculate globally weighted R² over all (image, target) pairs.

    Args:
        preds: Tensor of shape [N, 5] (model predictions)
        targets: Tensor of shape [N, 5] (true values)
        device: str, 'cpu' or 'xpu'

    Returns:
        r2_weighted: float (global weighted R²)
    """
    # Move to CPU and convert to numpy
    if preds.device != 'cpu':
        preds = preds.cpu()
    if targets.device != 'cpu':
        targets = targets.cpu()

    preds = preds.numpy()  # [N, 5]
    targets = targets.numpy()  # [N, 5]

    # Define weights per target column
    target_weights = {
        'Dry_Clover_g': 0.1,
        'Dry_Dead_g': 0.1,
        'Dry_Green_g': 0.1,
        'Dry_Total_g': 0.5,
        'GDM_g': 0.2
    }

    # Flatten: turn [N, 5] into [N*5] for global calculation
    n_samples = len(targets)
    flat_targets = targets.flatten()   # [N*5]
    flat_preds = preds.flatten()       # [N*5]

    # Create weight vector: repeat each target's weight N times
    weights = np.array([
        target_weights['Dry_Clover_g'],   # col 0
        target_weights['Dry_Dead_g'],     # col 1
        target_weights['Dry_Green_g'],    # col 2
        target_weights['Dry_Total_g'],    # col 3
        target_weights['GDM_g']           # col 4
    ])
    # Repeat weights for each sample → shape [N, 5] → then flatten
    flat_weights = np.tile(weights, n_samples)  # [N*5]

    # Compute weighted mean of true values
    y_bar_w = np.average(flat_targets, weights=flat_weights)

    # Compute weighted residual sum of squares (SS_res)
    ss_res = np.sum(flat_weights * (flat_targets - flat_preds) ** 2)

    # Compute weighted total sum of squares (SS_tot)
    ss_tot = np.sum(flat_weights * (flat_targets - y_bar_w) ** 2)

    # Avoid division by zero
    if ss_tot == 0:
        r2_weighted = 1.0 if ss_res == 0 else 0.0
    else:
        r2_weighted = 1 - (ss_res / ss_tot)

    return r2_weighted

Code to find the R^2 as detailed per the Kaggle page

In [ ]:
# Recreate the model architecture
model_eval = models.resnet18(weights=None)  # weights=None since we'll load ours
model_eval.fc = nn.Linear(model_eval.fc.in_features, 5)

# Load weights
model_eval.load_state_dict(torch.load("final_model.pth", map_location=device))
model_eval = model_eval.to(device)
model_eval.eval()  # set to evaluation mode

# Now evaluate on test set
test_loss = 0.0
all_preds = []      # ← collect all predictions
all_targets = []    # ← collect all targets
with torch.no_grad():
    for images, targets in test_loader:
        images, targets = images.to(device), targets.to(device)
        preds = model_eval(images)
        loss = criterion(preds, targets)
        test_loss += loss.item()
        all_preds.append(preds)
        all_targets.append(targets)

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss (from saved model): {avg_test_loss:.6f}")

# Concatenate all batches into single tensors
all_preds = torch.cat(all_preds, dim=0)      # [N_test, 5]
all_targets = torch.cat(all_targets, dim=0)  # [N_test, 5]
print(all_preds.shape, all_targets.shape)

# Calculate globally weighted R²
test_r2 = calculate_global_weighted_r2(all_preds, all_targets)
print(f"Test Weighted R²: {test_r2:.4f}")

Test Loss (from saved model): 265.493009
torch.Size([54, 5]) torch.Size([54, 5])
Test Weighted R²: 0.4558


First simple run thus far

In [2]:
!git clone https://github.com/prxshxntray/cnn-biomass-regression.git

Cloning into 'cnn-biomass-regression'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
